In [7]:
# Text clasification using pytorch
import torch
from torch import nn
import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split
import time


# Load data
df = pd.read_csv('spam_email.csv')
df.head()

# Split data into train, validation and test
train, test = train_test_split(df, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(train.shape, val.shape, test.shape)

train_data = [(text, label) for text, label in zip(train['text'], train['label'])]
val_data = [(text, label) for text, label in zip(val['text'], val['label'])]
test_data = [(text, label) for text, label in zip(test['text'], test['label'])]


# Tokenize data using get_tokenizer
tokenizer = get_tokenizer('basic_english')
train_iter = train['text'].apply(tokenizer).tolist()

# Build vocab
vocab = build_vocab_from_iterator(train_iter, specials=["<unk>", "<pad>"], special_first=True, min_freq=1)
vocab.set_default_index(vocab["<unk>"])
print(vocab(['here', 'is', 'an', 'example', 'bxyuwz']))

# Define text pipeline
text_pipeline = lambda x: [vocab[token] for token in tokenizer(x)]
label_pipeline = lambda x: 1 if x == '1' else 0

print(text_pipeline('here is an example'))
print(label_pipeline('1'))

# Generate data batch and iterator
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_text, _label) in batch:
        processed_text = text_pipeline(_text)
        if not processed_text:
            continue
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(processed_text, dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)



train_dataloader = DataLoader(train_data, batch_size=8, shuffle=False, collate_fn=collate_batch)
val_dataloader = DataLoader(val_data, batch_size=8, shuffle=False, collate_fn=collate_batch)
test_dataloader = DataLoader(test_data, batch_size=8, shuffle=False, collate_fn=collate_batch)


# Define model
class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=False)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)
    
# Define model
VOCAB_SIZE = len(vocab)
EMBED_DIM = 64
NUM_CLASS = 2
model = TextClassificationModel(VOCAB_SIZE, EMBED_DIM, NUM_CLASS).to(device)


# Define loss and optimizer
loss_fn = nn.CrossEntropyLoss()#.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=4.0)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)


# Train model
def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()
    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predited_label = model(text, offsets)
        loss = loss_fn(predited_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predited_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()


# Evaluate model
def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0
    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predited_label = model(text, offsets)
            loss = loss_fn(predited_label, label)
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count


# Train model
EPOCHS = 10
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    acc = evaluate(val_dataloader)
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | accuracy: {:8.3f} '.format(epoch,
                                            time.time() - epoch_start_time,
                                            acc))
    print('-' * 59)


(6400, 2) (1600, 2) (2000, 2)
[100, 15, 54, 1561, 43223]
[100, 15, 54, 1561]
1
| epoch   1 |   500/  800 batches | accuracy    1.000
-----------------------------------------------------------
| end of epoch   1 | time: 13.04s | accuracy:    1.000 
-----------------------------------------------------------
| epoch   2 |   500/  800 batches | accuracy    1.000
-----------------------------------------------------------
| end of epoch   2 | time: 13.04s | accuracy:    1.000 
-----------------------------------------------------------
| epoch   3 |   500/  800 batches | accuracy    1.000
-----------------------------------------------------------
| end of epoch   3 | time: 13.13s | accuracy:    1.000 
-----------------------------------------------------------
| epoch   4 |   500/  800 batches | accuracy    1.000
-----------------------------------------------------------
| end of epoch   4 | time: 13.98s | accuracy:    1.000 
-----------------------------------------------------------
|

In [14]:
# Test model
print('Checking the results of test dataset.')
acc = evaluate(test_dataloader)
print('test accuracy {:8.3f}'.format(acc))

# Predict

def predict(text, model, vocab, ngrams):
    tokenizer = get_tokenizer('basic_english')
    with torch.no_grad():
        text = torch.tensor([vocab[token] for token in tokenizer(text)], device=device)
        output = model(text, torch.tensor([0], device=device))
        return output.argmax(1).item()
    
ex_text_str = "Regarding your application, we regret to inform you that you have not been selected for the interview"
model = model.to("cpu")

print("This is a spam email" if predict(ex_text_str, model, vocab, 2) else "This is a ham email")

ex_text_str = "Get one free ticket by buying two tickets"
model = model.to("cpu")

print("This is a spam email" if predict(ex_text_str, model, vocab, 2) else "This is a ham email")

ex_text_str = "Hi, I am the prince of Nigeria. I want to transfer 1000000$ to your account. Please send me your account number"
model = model.to("cpu")

print("This is a spam email" if predict(ex_text_str, model, vocab, 2) else "This is a ham email")

# Print first 10 training samples
for i in range(10):
    print(train_data[i][1])


Checking the results of test dataset.
test accuracy    1.000
This is a ham email
This is a ham email
This is a ham email
1
1
1
0
0
0
1
0
1
1
